In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
from sklearn.model_selection import train_test_split
import numpy as np

# Загрузка данных из pickle-файлов
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

# Нормализация данных
X = X / 255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            # Создание модели
            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for _ in range(conv_layer - 1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            # Компиляция модели
            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'])

            # Разделение данных на обучающий и валидационный наборы
            X_train, X_val, y_train, y_val = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)

            # Обучение модели
            model.fit(X_train, y_train,
                      batch_size=32,
                      epochs=10,
                      validation_data=(X_val, y_val),
                      callbacks=[tensorboard])

            # Сохранение модели
            model.save('64x3-CNN.keras')

3-conv-64-nodes-0-dense-1710562308
Epoch 1/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.5702 - loss: 0.6682 - val_accuracy: 0.7007 - val_loss: 0.5668
Epoch 2/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.7221 - loss: 0.5524 - val_accuracy: 0.7525 - val_loss: 0.5051
Epoch 3/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.7648 - loss: 0.4901 - val_accuracy: 0.7692 - val_loss: 0.4863
Epoch 4/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.7900 - loss: 0.4490 - val_accuracy: 0.7949 - val_loss: 0.4561
Epoch 5/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8072 - loss: 0.4175 - val_accuracy: 0.7908 - val_loss: 0.4619
Epoch 6/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8216 - loss: 0.3937 - val_accuracy: 0.7988 - val_loss: 0.4352
Epoch 7/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8377 - loss: 0.3642 - val_accuracy: 0.7946 - val_loss: 0.4398
Epoch 8/10
546/546 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step -

In [10]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

In [11]:
model = tf.keras.models.load_model("64x3-CNN.keras")

In [12]:
prediction = model.predict([prepare('10000.jpg')])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


In [14]:
print(CATEGORIES[int(prediction[0][0])])

Dog


In [42]:
prediction = model.predict([prepare('11317.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[1.]]
Cat


In [51]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.keras")

prediction = model.predict([prepare('11317.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[[0. 1.]]
Dog


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Dog: 100.0%
Cat: 100.0%
